In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'purchased_all_food'
AUTHOR = 'will'

In [3]:
task = Task(
    id=TASK_ID,
    category="First-order logic",
    name="People that purchased all possible items",
    description="Find all buyers that ordered every food item at least once.",
    plan=[
        Plan(id="iter", description="all buyers"),
        Plan(id="ordered", description="ordered"),
        Plan(id="all", description="every food item at least once")
    ],
    sample_input={
        "food": [
            {"id": 1, "name": "Burrito"},
            {"id": 2, "name": "Sushi"}
        ],
        "orders": [
            {"id": 1, "buyer": "Will", "food": 1},
            {"id": 2, "buyer": "Scott", "food": 2},
            {"id": 3, "buyer": "Scott", "food": 2},
            {"id": 4, "buyer": "Will", "food": 2},
        ]
    },
    sample_output=["Will"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [15]:
q = replace(prototype,
    language='q',
    source='''buyers: `buyer xgroup orders;
total: count food;
purchased_all_food:
  (where {(count distinct x[`food]) = total} each buyers) `buyer''').load_plan()
q.execute(task)
q.save()

In [16]:
q.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "q", "plan": {}, "source": "buyers: `buyer xgro…

In [13]:
r = replace(prototype,
    language='r',
    source='''purchased_all_food <- function(food, orders) {
  n_food <- count(food)    
  orders %>%
    group_by(buyer) %>%
    distinct(food) %>%
    count() %>%
    filter(n == n_food) %>%
    pull(buyer)
}''').load_plan()
r.execute(task)
r.save()

In [14]:
r.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "r", "plan": {}, "source": "purchased_all_food …

In [7]:
sql = replace(prototype,
    language='sql',
    source='''SELECT DISTINCT buyer
FROM orders o1
WHERE 
  (SELECT COUNT(DISTINCT food) FROM orders o2 
   WHERE o1.buyer = o2.buyer) = 
  (SELECT COUNT(*) FROM food)''').load_plan()
sql.execute(task)
sql.save()

In [8]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "sql", "plan": {}, "source": "SELECT DISTINCT b…

In [14]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl has_purchased(Buyer:symbol, Food:number)
has_purchased(Buyer, Food) :- orders(Buyer, Food, _).

purchased_all_food(Buyer) :-
  orders(Buyer, _, _),
  N_food = count : food(_, _),
  N_unique_orders = count : has_purchased(Buyer, _),
  N_food = N_unique_orders.''').load_plan()
datalog.execute(task)
datalog.save()

In [15]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "datalog", "plan": {}, "source": ".decl has_pur…

In [18]:
pandas = replace(prototype,
    language='python-pandas',
    source='''def purchased_all_food(food, orders):
  n_food = len(food)
  n_unique_orders = (orders
    .groupby('buyer')
    .food.unique() 
    .map(len))
  return n_unique_orders[n_unique_orders == n_food].index.values''').load_plan()
pandas.execute(task)
pandas.save()

In [19]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "python-pandas", "plan": {}, "source": "def pur…

In [23]:
imperative = replace(prototype,
    language='python-imperative',
    source='''def purchased_all_food(food, orders):
  n_food = len(food)
  unique_orders = defaultdict(set)
  for order in orders:
    unique_orders[order["buyer"]].add(order["food"])
      
  buyers = []
  for buyer, orders in unique_orders.items():
    if len(orders) == n_food:
      buyers.append(buyer)
  return buyers''').load_plan()
imperative.execute(task)
imperative.save()

In [24]:
imperative.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "python-imperative", "plan": {}, "source": "def…

In [25]:
functional = replace(prototype,
    language='python-functional',
    source='''def purchased_all_food(food, orders):
    n_food = len(food)
    buyers = set([order["buyer"] for order in orders])
    return [
        buyer
        for buyer in buyers
        if len(set([order["food"] for order in orders 
             if order["buyer"] == buyer])) == n_food
    ]''').load_plan()
functional.execute(task)
functional.save()

In [26]:
functional.widget(task)

Output()

CodeWidget(program='{"task": "purchased_all_food", "language": "python-functional", "plan": {}, "source": "def…